In [ ]:
# Import necessary libraries
import pandas as pd
from sklearn.linear_model import LinearRegression
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [ ]:
# Load training and test datasets
train_df = pd.read_csv('drive/MyDrive/Colab Notebooks/amazon_train.csv')
test_df = pd.read_csv('drive/MyDrive/Colab Notebooks/amazon_test.csv')

In [ ]:
# Perform one-hot encoding for the 'category' feature in the training dataset
one_hot_encoding_train = pd.get_dummies(train_df.category, prefix='category')
train_df = train_df.join(one_hot_encoding_train)

# Perform one-hot encoding for the 'category' feature in the test dataset
one_hot_encoding_test = pd.get_dummies(test_df.category, prefix='category')
test_df = test_df.join(one_hot_encoding_test)

In [ ]:
from datetime import datetime

# Convert 'unixReviewTime' to datetime and extract date-time features in the training dataset
train_df['datetime'] = pd.to_datetime(train_df['unixReviewTime'], unit='s')
train_df['hour'] = train_df['datetime'].dt.hour
train_df['weekday'] = train_df['datetime'].dt.dayofweek
train_df['month'] = train_df['datetime'].dt.month
train_df['year'] = train_df['datetime'].dt.year

# Convert 'unixReviewTime' to datetime and extract date-time features in the test dataset
test_df['datetime'] = pd.to_datetime(test_df['unixReviewTime'], unit='s')
test_df['hour'] = test_df['datetime'].dt.hour
test_df['weekday'] = test_df['datetime'].dt.dayofweek
test_df['month'] = test_df['datetime'].dt.month
test_df['year'] = test_df['datetime'].dt.year


In [ ]:
# Mapping 'category' to numerical values in the training dataset
category_mapping = {
    'automotive': 1,
    'CDs': 2,
    'grocery': 3,
    'cell_phones': 4,
    'sports': 5,
    'toys': 6
}
train_df['category_no'] = train_df['category'].replace(category_mapping)

# Mapping 'category' to numerical values in the test dataset
test_df['category_no'] = test_df['category'].replace(category_mapping)

In [ ]:
# Creating 'combined_text' by concatenating 'reviewText' and 'summary' in the training dataset
train_df['combined_text'] = train_df['reviewText'] + ' ' + train_df['summary']

# Creating 'combined_text' by concatenating 'reviewText' and 'summary' in the test dataset
test_df['combined_text'] = test_df['reviewText'] + ' ' + test_df['summary']

In [ ]:
# Mapping 'verified' column to binary values (0 and 1) in the training dataset
train_df['verified'] = train_df['verified'].map({False: 0, True: 1})

# Mapping 'verified' column to binary values (0 and 1) in the test dataset
test_df['verified'] = test_df['verified'].map({False: 0, True: 1})

In [ ]:
import nltk
from nltk.corpus import stopwords

# Filling NaN with empty string in text columns
train_df['reviewText'] = train_df['reviewText'].fillna('')
train_df['summary'] = train_df['summary'].fillna('')

# Downloading stopwords from NLTK
nltk.download('stopwords')

# Function to remove stopwords from text
def remove_stopwords(text):
    if isinstance(text, str):
        stop_words = set(stopwords.words('english'))
        cleaned_text = ' '.join([word for word in text.split() if word.lower() not in stop_words])
        return cleaned_text
    else:
        return text

# Applying remove_stopwords function to 'reviewText' and 'summary' columns in the training dataset
train_df['reviewText'] = train_df['reviewText'].apply(remove_stopwords)
train_df['summary'] = train_df['summary'].apply(remove_stopwords)

# Applying remove_stopwords function to 'reviewText' and 'summary' columns in the test dataset
test_df['reviewText'] = test_df['reviewText'].apply(remove_stopwords)
test_df['summary'] = test_df['summary'].apply(remove_stopwords)

# Displaying the head of the training dataset
train_df.head()

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,overall,verified,reviewTime,reviewerID,asin,reviewerName,reviewText,summary,unixReviewTime,vote,...,category_grocery,category_sports,category_toys,datetime,hour,weekday,month,year,category_no,combined_text
0,1,0,"11 12, 2016",C413C78E10E54C5DB41837889F36C1E8,565D194F38B1CC3F806EE677C61F639C,465E154EC79AFFAB5EB2607198B21433,reviews product fake.,"fake reviews, beware.",1478908800,2.0,...,0,0,0,2016-11-12,0,5,11,2016,1,all of the reviews for this product are fake. ...
1,1,1,"12 6, 2016",490AE37808EFEE3AF4FE6DEBDEB5A4C8,0D66512A0A7F580523AB996378DF0F14,760C63E8E5E8DC3FAA01878D37BA5678,wrong part. fault.,One Star,1480982400,NaN,...,0,0,0,2016-12-06,0,1,12,2016,1,wrong part. our fault. One Star
2,1,1,"09 17, 2014",74A9FA5A64449BEE2A2E8E3F62872F0F,A0E45600FF2C5A779CB4314F379C253A,C6E4DD5C1C4EC09E90182644ED6CA9EF,wire set really sucks!!!,One Star,1410912000,NaN,...,0,0,0,2014-09-17,0,2,9,2014,1,this wire set it really sucks!!! One Star
3,1,1,"06 11, 2016",EB561158A2829D98B467FE03CC1E45F1,37AB9A82470595E0ACB88BAC48C150EE,F4892A77EA45C52F40AB17ED537EF9FF,"first use, leaked instantly. even 5 bucks wort...",One Star,1465603200,NaN,...,0,0,0,2016-06-11,0,5,6,2016,1,"first use, it leaked instantly. even at 5 buck..."
4,1,1,"12 23, 2017",5045D801332850D21618DD13A697CD9B,5772FF30428EEB8E0258C1A53CA2EC50,522F0BBFF2B47F1D63FF781A0AB1D079,fit,One Star,1513987200,NaN,...,0,0,0,2017-12-23,0,5,12,2017,1,didn't fit One Star


*Sentiment Analysis*

In [ ]:
# Downloading VADER lexicon from NLTK
nltk.download('vader_lexicon')
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
import pandas as pd

# Initialize the VADER sentiment analyzer
sia = SentimentIntensityAnalyzer()

# Calculate sentiment scores for 'reviewText'
train_df['reviewText_sentiment'] = train_df['reviewText'].apply(lambda x: sia.polarity_scores(x))
test_df['reviewText_sentiment'] = test_df['reviewText'].apply(lambda x: sia.polarity_scores(x))

# Calculate sentiment scores for 'summary'
train_df['summary_sentiment'] = train_df['summary'].apply(lambda x: sia.polarity_scores(x))
test_df['summary_sentiment'] = test_df['summary'].apply(lambda x: sia.polarity_scores(x) if not pd.isnull(x) else np.nan)

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
# Split 'reviewText_sentiment' into separate columns
train_df['reviewText_negative'] = train_df['reviewText_sentiment'].apply(lambda x: x['neg'])
train_df['reviewText_neutral'] = train_df['reviewText_sentiment'].apply(lambda x: x['neu'])
train_df['reviewText_positive'] = train_df['reviewText_sentiment'].apply(lambda x: x['pos'])

test_df['reviewText_negative'] = test_df['reviewText_sentiment'].apply(lambda x: x['neg'])
test_df['reviewText_neutral'] = test_df['reviewText_sentiment'].apply(lambda x: x['neu'])
test_df['reviewText_positive'] = test_df['reviewText_sentiment'].apply(lambda x: x['pos'])

# Split 'summary_sentiment' into separate columns
train_df['summary_negative'] = train_df['summary_sentiment'].apply(lambda x: x['neg'])
train_df['summary_neutral'] = train_df['summary_sentiment'].apply(lambda x: x['neu'])
train_df['summary_positive'] = train_df['summary_sentiment'].apply(lambda x: x['pos'])

# Extract sentiment scores while replacing NaN with 0
test_df['summary_negative'] = test_df['summary_sentiment'].apply(lambda x: x['neg'] if not pd.isnull(x) else 0)
test_df['summary_neutral'] = test_df['summary_sentiment'].apply(lambda x: x['neu'] if not pd.isnull(x) else 0)
test_df['summary_positive'] = test_df['summary_sentiment'].apply(lambda x: x['pos'] if not pd.isnull(x) else 0)

In [ ]:
train_df['reviewText'] = train_df['reviewText'].fillna('')
train_df['summary'] = train_df['summary'].fillna('')
train_df['combined_text'] = train_df['combined_text'].fillna('')

# Fill NaN with empty string in text columns
test_df['summary'] = test_df['summary'].fillna('')
test_df['reviewText'] = test_df['reviewText'].fillna('')
test_df['combined_text'] = test_df['combined_text'].fillna('')

# Replace NaN with 0 in numeric columns
numeric_columns = ['reviewText_negative', 'reviewText_neutral', 'reviewText_positive', 'summary_negative', 'summary_neutral', 'summary_positive']
test_df[numeric_columns] = test_df[numeric_columns].fillna(0)

In [ ]:
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, roc_auc_score


train_df['reviewText'] = train_df['reviewText'].fillna('')
train_df['summary'] = train_df['summary'].fillna('')
train_df['combined_text'] = train_df['combined_text'].fillna('')

X = train_df[['category_no', 'verified', 'weekday', 'reviewText_negative', 'reviewText_neutral', 'reviewText_positive', 'summary_negative', 'summary_neutral', 'summary_positive', 'combined_text']]
y = train_df['overall']

y = y - 1
vectorizer1 = TfidfVectorizer()
vectorizer2 = TfidfVectorizer()
vectorizer3 = TfidfVectorizer()

column_transformer = ColumnTransformer([
    ('tfidf3', vectorizer3, 'combined_text')],
    remainder='passthrough')

# Splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Fit and transform the transformer on the training data
X_train_transformed = column_transformer.fit_transform(X_train)
X_test_transformed = column_transformer.transform(X_test)



# Decision Tree
model_tree = DecisionTreeClassifier()
pipe_tree = Pipeline(steps=[('tfidf', column_transformer), ('classify', model_tree)])
pipe_tree.fit(X_train, y_train)
y_pred_tree = pipe_tree.predict(X_test)
f1_tree = f1_score(y_test, y_pred_tree, average='weighted')
conf_matrix_tree = confusion_matrix(y_test, y_pred_tree)
acc_tree = accuracy_score(y_test, y_pred_tree)
roc_auc_tree = roc_auc_score(y_test, pipe_tree.predict_proba(X_test), multi_class='ovr')


print("Decision Tree:")
print(f"F1 Score - {f1_tree}, Accuracy - {acc_tree}, ROC AUC - {roc_auc_tree}")
print("Confusion Matrix:")
print(conf_matrix_tree)

# Random Forest
model_forest = RandomForestClassifier()
pipe_forest = Pipeline(steps=[('tfidf', column_transformer), ('classify', model_forest)])
pipe_forest.fit(X_train, y_train)
y_pred_forest = pipe_forest.predict(X_test)
f1_forest = f1_score(y_test, y_pred_forest, average='weighted')
conf_matrix_forest = confusion_matrix(y_test, y_pred_forest)
acc_forest = accuracy_score(y_test, y_pred_forest)
roc_auc_forest = roc_auc_score(y_test, pipe_forest.predict_proba(X_test), multi_class='ovr')



print("Random Forest:")
print(f"F1 Score - {f1_forest}, Accuracy - {acc_forest}, ROC AUC - {roc_auc_forest}")
print("Confusion Matrix:")
print(conf_matrix_forest)


# XGBoost
model_xgb = XGBClassifier()
pipe_xgb = Pipeline(steps=[('tfidf', column_transformer), ('classify', model_xgb)])
pipe_xgb.fit(X_train, y_train)
y_pred_xgb = pipe_xgb.predict(X_test)
f1_xgb = f1_score(y_test, y_pred_xgb, average='weighted')
conf_matrix_xgb = confusion_matrix(y_test, y_pred_xgb)
acc_xgb = accuracy_score(y_test, y_pred_xgb)
roc_auc_xgb = roc_auc_score(y_test, pipe_xgb.predict_proba(X_test), multi_class='ovr')


print("XGBoost:")
print(f"F1 Score - {f1_xgb}, Accuracy - {acc_xgb}, ROC AUC - {roc_auc_xgb}")
print("Confusion Matrix:")
print(conf_matrix_xgb)

Decision Tree:
F1 Score - 0.45839794074588636, Accuracy - 0.4580335731414868, ROC AUC - 0.6616666345736525
Confusion Matrix:
[[585 262 166 111  68]
 [286 460 210 146  90]
 [167 222 488 198  97]
 [ 98 162 188 505 201]
 [ 61  71 113 247 636]]
Random Forest:
F1 Score - 0.5432732677620201, Accuracy - 0.5469338814662555, ROC AUC - 0.8404928195636246
Confusion Matrix:
[[869 204  65  28  26]
 [389 484 214  75  30]
 [162 237 536 183  54]
 [ 65 138 199 554 198]
 [ 55  45  73 205 750]]
XGBoost:
F1 Score - 0.5715283021660719, Accuracy - 0.5719424460431655, ROC AUC - 0.8644469483506164
Confusion Matrix:
[[779 254  85  47  27]
 [286 555 229  92  30]
 [109 225 586 201  51]
 [ 39  88 183 632 212]
 [ 32  40  62 207 787]]


In [ ]:
# Perform prediction
predicted_classes = pipe_xgb.predict(test_df[['category_no', 'verified', 'weekday', 'reviewText_negative', 'reviewText_neutral', 'reviewText_positive', 'summary_negative', 'summary_neutral', 'summary_positive', 'combined_text']])

# Combine IDs with predicted classifications
results = pd.DataFrame({'id': test_df['id'], 'pred': predicted_classes+1})

# Display the results
print(results)

         id  pred
0        a0     1
1        a1     1
2        a2     1
3        a3     2
4        a4     1
...     ...   ...
4495  a4495     5
4496  a4496     5
4497  a4497     5
4498  a4498     5
4499  a4499     4

[4500 rows x 2 columns]


In [ ]:
#results[["pred"]] = results[["pred"]].apply(pd.to_numeric)
results.to_csv("sample_data/results.csv", index = False)